### 핀테크 뉴스 수집

1. 핀테크 지원센터 홈페이지 → 알림마당 → 핀테크 정보 → 핀테크 뉴스에 접속
https://fintech.or.kr/web/board/boardContentsListPage.do?board_id=6&menu_id=6501&miv_pageNo=
2. 일자별로 있는 [11/18 핀테크 주요뉴스] 안에 있는 뉴스 제목과 원문 link 수집
3. 원문링크로 들어가서 뉴스 본문을 2025년 1월 2일까지 수집 후(약 18page) DB에 저장
4. 데이터 베이스에 저장된 뉴스를 데이터프레임으로 불러와서 텍스트 전처리, Wordcloud, LDA 토픽 모델링
5. 2025년 핀테크 뉴스에서 가장 많이 다룬 주제를 찾고 서비스 기획 아이디어 제시하기

In [6]:
import requests 
import pandas as pd
import time
import os
import re
from bs4 import BeautifulSoup as bs

In [7]:
def text_clean(text):
    # HTML 태그를 없애는 정규표현식
    result = re.sub(r"</?[^>]+>", "", text)
    # 한글, 영문, 숫자 외의 모든 문자 제거 후 공백으로 변환
    result = re.sub(r"[^가-힣a-zA-Z0-9]", " ", result)
    result = result.replace("  ", " ").replace("  ", " ").replace("  ", " ")
    return result

In [8]:
url = "https://fintech.or.kr/web/board/boardContentsListPage.do"
payload = dict(board_id = "6", menu_id = "6501", miv_pageNo="1")
r = requests.get(url, params=payload)
print(r.url)
print(r.status_code)
response = bs(r.content, 'lxml')
response

https://fintech.or.kr/web/board/boardContentsListPage.do?board_id=6&menu_id=6501&miv_pageNo=1
200


<!DOCTYPE html>
<html lang="ko" xml:lang="ko">
<head>
<title>
		
		
		핀테크 포털 - 한국핀테크지원센터
		
		</title>
<meta content="핀테크 생태계 활성화로 금융의 혁신과 성장을 지원합니다.." name="description"/>
<meta content="핀테크 포털 - 한국핀테크지원센터" name="keywords"/>
<meta content="http://fintech.or.kr/images/preview_page.png" property="og:image"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1.0,maximum-scale=1.0,minimum-scale=1.0,user-scalable=no" name="viewport"/>
<meta content="XpressEngine" name="Generator"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="o8yb702i79l2intg1e9ov2hyavzbji" name="facebook-domain-verification"/>
<!-- <link rel="shortcut icon" href="/images/web/favicon.ico" type="image/x-icon"> -->
<!-- <link rel="icon" href="/images/web/favicon.ico" type="image/x-icon"> -->
<link href="/css/web/default.css?ver=20200916" rel="stylesheet" type="text/css"/>
<link href="/css/web/reset.css?ver=20210629" rel="stylesheet" t

In [17]:
news_list = response.select("a.txtl")
news_list

[]